In [ ]:
import geopandas as gpd
import requests # to make API calls
import os
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from pyproj import CRS, Transformer

load_dotenv()  # Load environment variables from .env file
mapquest_API_key = os.getenv("mapquest_API_key")

In [ ]:
import plotly.express as px

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})


In [ ]:
df

In [ ]:

import plotly.express as px

fig = px.choropleth_map(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           map_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )


In [ ]:


fig.show()

In [ ]:
 # Copernicus Urban Atlas WMS/WFS service endpoints
wfs_url = "https://image.discomap.eea.europa.eu/arcgis/services/UrbanAtlas/UA_UrbanAtlas_2018/MapServer/WFSServer"
wms_url = "https://image.discomap.eea.europa.eu/arcgis/services/UrbanAtlas/UA_UrbanAtlas_2018/MapServer/WMSServer"


In [ ]:
gdf = gpd.read_file('/Users/annaandersson/projects_personal/newnewExeter/data/Results/UK018L3_EXETER_UA2018_v013/Data/UK018L3_EXETER_UA2018_v013.gpkg')

In [ ]:
# 1. Define Exeter city center (approx)
# You can adjust this if you need a more precise point (Exeter Cathedral is 50.722, -3.533)
exeter_lon, exeter_lat = -3.533, 50.722  

# 2. Create a Point geometry (in WGS84)
center_point = Point(exeter_lon, exeter_lat)

# 3. Define coordinate reference systems
crs_wgs84 = CRS("EPSG:4326")  # lat/lon
crs_uk = CRS("EPSG:27700")    # British National Grid, meters

# 4. Transform point to projected CRS (so buffer is in meters)
transformer = Transformer.from_crs(crs_wgs84, crs_uk, always_xy=True)
x, y = transformer.transform(exeter_lon, exeter_lat)
point_projected = Point(x, y)

# 5. Create a square polygon (2 km side → 1 km half-side)
half_side = 1000  # meters
square = Polygon([
    (x - half_side, y - half_side),
    (x + half_side, y - half_side),
    (x + half_side, y + half_side),
    (x - half_side, y + half_side)
])

# 6. Create GeoDataFrame and reproject back to WGS84 if you want to export/view on a map
gdf_square = gpd.GeoDataFrame({"name": ["Exeter 2km Square"]}, geometry=[square], crs=crs_uk)
gdf_square_wgs84 = gdf_square.to_crs(epsg=4326)


In [ ]:
gdf = gdf.to_crs(gdf_square.crs)

# 2️⃣ Spatial join to keep only intersecting geometries
gdf_intersections = gpd.sjoin(gdf, gdf_square, how="inner", predicate="intersects")


In [ ]:
gdf_intersections

In [ ]:
gdf.columns

In [ ]:
gdf.class_2018.unique()

In [ ]:
bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

In [ ]:
gdf.geometry

In [ ]:
gdf.geometry[1]

In [ ]:
gdf25 = gdf.head(5).copy()

In [ ]:
fig = px.choropleth_mapbox(
        gdf25,
        geojson=gdf25.geometry,
        locations=gdf25.index,
        color='code_2018',
        hover_data=['class_2018'],
        mapbox_style="open-street-map",
        zoom=11,
        center={"lat": center_lat, "lon": center_lon},
        opacity=0.7,
        title=f"Exeter Urban Atlas - Land Use Classification ({'class_2018'})"
    )

In [ ]:

go_fig = create_go_multipolygon_map(gdf_intersections, 'code_2018')
go_fig.show()


In [ ]:
# Alternative approach: Use Plotly Graph Objects for better multipolygon control
def create_go_multipolygon_map(gdf, color_column='code_2018'):
    """
    Create map using Plotly Graph Objects for better multipolygon handling
    """
    
    # Ensure WGS84 projection
    if gdf.crs != 'EPSG:4326':
        gdf_plot = gdf.to_crs('EPSG:4326')
    else:
        gdf_plot = gdf.copy()
    
    # Get unique values for color mapping
    unique_values = gdf_plot[color_column].unique()
    colors = px.colors.qualitative.Set3 + px.colors.qualitative.Pastel + px.colors.qualitative.Dark2
    
    # Create color mapping
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    
    # Get map bounds
    bounds = gdf_plot.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Create figure
    fig = go.Figure()
    
    # Add each unique class as a separate trace
    for value in unique_values:
        subset = gdf_plot[gdf_plot[color_column] == value]
        
        if len(subset) > 0:
            # Convert subset to GeoJSON
            geojson_subset = json.loads(subset.to_json())
            
            # Add trace for this class
            fig.add_trace(go.Choroplethmapbox(
                geojson=geojson_subset,
                locations=list(range(len(subset))),
                z=[1] * len(subset),  # Dummy values since we're using custom colors
                colorscale=[[0, color_map[value]], [1, color_map[value]]],
                showscale=False,
                marker_opacity=0.7,
                marker_line_width=1,
                marker_line_color='white',
                name=f"{color_column}: {value}",
                hovertemplate=f"<b>{color_column}:</b> {value}<br>" +
                             "<b>Class:</b> %{customdata}<br>" +
                             "<extra></extra>",
                customdata=subset['class_2018'].values if 'class_2018' in subset.columns else [f"Code {value}"] * len(subset)
            ))
    
    # Update layout
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(
            center=dict(lat=center_lat, lon=center_lon),
            zoom=12
        ),
        margin={"r":0,"t":50,"l":0,"b":0},
        height=800,
        title=f"Exeter Urban Atlas - {color_column} (Graph Objects Method)",
        showlegend=True
    )
    
    return fig

